In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [5]:
old_play_data_2022 = pd.read_csv("Raw_2022_nfl_data_play.csv")
old_play_data_2023 = pd.read_csv("Raw_2023_nfl_data_play.csv")
old_nfl_scores_2022 = pd.read_csv("nfl_2022_scores_csv.csv")
old_nfl_scores_2023 = pd.read_csv("nfl_2023_scores_csv.csv")

In [7]:
old_play_data_2022.head()

GameId    GameDate  Quarter  Minute  Second OffenseTeam DefenseTeam  \
0  2022091106  2022-09-11        2       1      17         MIA          NE   
1  2022091106  2022-09-11        3       7       4          NE         MIA   
2  2022091106  2022-09-11        3       6      57          NE         MIA   
3  2022091106  2022-09-11        3       6      17          NE         MIA   
4  2022091106  2022-09-11        4       9      46          NE         MIA   

   Down  ToGo  YardLine  ...  IsTwoPointConversion  \
0     2    10        55  ...                     0   
1     4     1        72  ...                     0   
2     1    10        77  ...                     0   
3     2    10        77  ...                     0   
4     4     3       100  ...                     0   

   IsTwoPointConversionSuccessful  RushDirection  YardLineFixed  \
0                               0            NaN             45   
1                               0            NaN             28   
2                               0         CENTER             23   
3                               0            NaN             23   
4                               0            NaN              0   

  YardLineDirection  IsPenaltyAccepted  PenaltyTeam  IsNoPlay  \
0               OPP                  0          NaN         0   
1               OPP                  1          MIA         1   
2               OPP                  0          NaN         0   
3               OPP                  0          NaN         0   
4               OPP                  0           NE         0   

         PenaltyType  PenaltyYards  
0                NaN             0  
1    ILLEGAL CONTACT             5  
2                NaN             0  
3                NaN             0  
4  OFFENSIVE HOLDING             0  

[5 rows x 45 columns]

In [9]:
print(old_play_data_2022.columns)

Index(['GameId', 'GameDate', 'Quarter', 'Minute', 'Second', 'OffenseTeam',
       'DefenseTeam', 'Down', 'ToGo', 'YardLine', 'Unnamed: 10',
       'SeriesFirstDown', 'Unnamed: 12', 'NextScore', 'Description', 'TeamWin',
       'Unnamed: 16', 'Unnamed: 17', 'SeasonYear', 'Yards', 'Formation',
       'PlayType', 'IsRush', 'IsPass', 'IsIncomplete', 'IsTouchdown',
       'PassType', 'IsSack', 'IsChallenge', 'IsChallengeReversed',
       'Challenger', 'IsMeasurement', 'IsInterception', 'IsFumble',
       'IsPenalty', 'IsTwoPointConversion', 'IsTwoPointConversionSuccessful',
       'RushDirection', 'YardLineFixed', 'YardLineDirection',
       'IsPenaltyAccepted', 'PenaltyTeam', 'IsNoPlay', 'PenaltyType',
       'PenaltyYards'],
      dtype='object')


In [11]:
old_nfl_scores_2022.head()

Week        Date              Visitor  VisitorScore                Home  \
0  Week 1  09/08/2022        Buffalo Bills            31    Los Angeles Rams   
1  Week 1  09/11/2022   Indianapolis Colts            20      Houston Texans   
2  Week 1  09/11/2022  San Francisco 49ers            10       Chicago Bears   
3  Week 1  09/11/2022  Pittsburgh Steelers            23  Cincinnati Bengals   
4  Week 1  09/11/2022  Philadelphia Eagles            38       Detroit Lions   

   HomeScore   OT  
0         10  NaN  
1         20   OT  
2         19  NaN  
3         20   OT  
4         35  NaN

In [13]:
print(old_nfl_scores_2022.columns)

Index(['Week', 'Date', 'Visitor', 'VisitorScore', 'Home', 'HomeScore', 'OT'], dtype='object')


Team Feature Extraction

In [25]:
import pandas as pd
import numpy as np

# Load the datasets for the 2022 and 2023 seasons that include a "HomeWon" column.
# This column indicates if the home team won the game (1 for win, 0 for loss).

# NOTE: You'll probably have to change these file paths to the paths for your environment
# (click folder icon on left, drag and drop them in, and then right click them to get the path and paste them in the quotes below)
data_2022_updated = pd.read_csv("BEST_2022_nfl_data_play_with_scores_and_HomeWon.csv")
data_2023_updated = pd.read_csv("BEST_2023_nfl_data_play_with_scores_and_HomeWon.csv")

# Load the dataset containing the upcoming games schedule.
upcoming_games = pd.read_csv("BEST New_Upcoming_Schedule.csv")

# Combine the data from the 2022 and 2023 seasons into a single DataFrame.
#all_data = pd.concat([data_2022_updated, data_2023_updated])
all_data = data_2023_updated

# 1. Average Points Scored
# Calculate the average points scored by the home and visitor teams.
avg_points_scored_home = all_data.groupby('Home')['HomeScore'].mean()
avg_points_scored_visitor = all_data.groupby('Visitor')['VisitorScore'].mean()

# 2. Average Points Allowed
# Calculate the average points allowed by the home and visitor teams.
avg_points_allowed_home = all_data.groupby('Home')['VisitorScore'].mean()
avg_points_allowed_visitor = all_data.groupby('Visitor')['HomeScore'].mean()

# Calculate the overall average points scored and allowed by combining the home and visitor averages.
overall_avg_points_scored = (avg_points_scored_home + avg_points_scored_visitor) / 2
overall_avg_points_allowed = (avg_points_allowed_home + avg_points_allowed_visitor) / 2

# 3. Win Rate
# Calculate the total number of wins for home and visitor teams.
home_wins = all_data.groupby('Home')['HomeWon'].sum()
visitor_wins = all_data.groupby('Visitor').apply(lambda x: len(x) - x['HomeWon'].sum())

# Calculate the total number of games played by each team as home and visitor.
total_games_home = all_data['Home'].value_counts()
total_games_visitor = all_data['Visitor'].value_counts()

# Calculate the overall number of wins and total games played by each team.
overall_wins = home_wins + visitor_wins
total_games = total_games_home + total_games_visitor

# Calculate the win rate for each team.
win_rate = overall_wins / total_games

# Calculate the average outcome of games between each pair of teams (home vs visitor).
# head_to_head = all_data.groupby(['Home', 'Visitor'])['HomeWon'].mean()

# Create a new data frame to store the features for each team.
team_features = pd.DataFrame({
    'AvgPointsScored': overall_avg_points_scored,
    'AvgPointsAllowed': overall_avg_points_allowed,
    'WinRate': win_rate
})

# Reset the index of the team_features DataFrame and rename the index column to "Team".
team_features.reset_index(inplace=True)
team_features.rename(columns={'Home': 'Team'}, inplace=True)

# Display the first few rows of the team_features DataFrame.
team_features.head()

C:\Users\shash\AppData\Local\Temp\ipykernel_2008\272327803.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  visitor_wins = all_data.groupby('Visitor').apply(lambda x: len(x) - x['HomeWon'].sum())


Team  AvgPointsScored  AvgPointsAllowed   WinRate
0  ARI        22.000000         25.355769  0.260526
1  ATL        15.492551         19.294365  0.500675
2  BAL        24.633973         14.814507  0.713392
3  BUF        34.659834         13.942320  0.739255
4  CAR        17.259582         25.820934  0.000000

In [29]:
upcoming_games.head()

Home Visitor
0  CHI     WAS
1  HOU     ATL
2  CAR     DET
3   NO      NE
4  CIN     ARI

In [31]:
# Calculate defensive features for each NFL team.

# 1. Average points defended:
# This metric is essentially the same as AvgPointsAllowed, which we already computed in previous steps so we won't recompute it here.

# 2. Average conceded plays:
# A play is considered successful for the offense if it results in a touchdown or doesn't result in a turnover.
# Create a new column 'SuccessfulPlay' in the all_data DataFrame to represent this.
all_data['SuccessfulPlay'] = all_data['IsTouchdown'] | (~all_data['IsInterception'] & ~all_data['IsFumble'])

# Calculate the average rate of successful plays conceded when playing at home.
avg_conceded_plays_home = all_data.groupby('Home')['SuccessfulPlay'].mean()

# Calculate the average rate of successful plays conceded when playing as a visitor.
avg_conceded_plays_visitor = all_data.groupby('Visitor')['SuccessfulPlay'].mean()

# Calculate the overall average rate of successful plays conceded for each team.
overall_avg_conceded_plays = (avg_conceded_plays_home + avg_conceded_plays_visitor) / 2

# 3. Average forced turnovers:
# Create a new column 'Turnover' that indicates if a play resulted in a turnover (either interception or fumble).
all_data['Turnover'] = all_data['IsInterception'] | all_data['IsFumble']

# Calculate the average rate of turnovers forced when playing at home.
avg_forced_turnovers_home = all_data.groupby('Home')['Turnover'].mean()

# Calculate the average rate of turnovers forced when playing as a visitor.
avg_forced_turnovers_visitor = all_data.groupby('Visitor')['Turnover'].mean()

# Calculate the overall average rate of turnovers forced for each team.
overall_avg_forced_turnovers = (avg_forced_turnovers_home + avg_forced_turnovers_visitor) / 2

# Create a new DataFrame to store the defensive features for each team.
team_features_defensive = pd.DataFrame({
    'Team': team_features['Team'].values,
    'AvgPointsDefended': team_features['AvgPointsAllowed'].values,
    'AvgConcededPlays': overall_avg_conceded_plays.values,
    'AvgForcedTurnovers': overall_avg_forced_turnovers.values
})

# Merge the defensive features with the original team features to create a combined DataFrame.
team_features_combined = team_features.merge(team_features_defensive, on='Team')

# Display the first few rows of the combined team features DataFrame.
team_features_combined.head()


Team  AvgPointsScored  AvgPointsAllowed   WinRate  AvgPointsDefended  \
0  ARI        22.000000         25.355769  0.260526          25.355769   
1  ATL        15.492551         19.294365  0.500675          19.294365   
2  BAL        24.633973         14.814507  0.713392          14.814507   
3  BUF        34.659834         13.942320  0.739255          13.942320   
4  CAR        17.259582         25.820934  0.000000          25.820934   

   AvgConcededPlays  AvgForcedTurnovers  
0         -1.011919            0.013292  
1         -1.017538            0.018890  
2         -1.021287            0.021287  
3         -1.030072            0.031484  
4         -1.018031            0.020741

In [33]:
# Calculate additional offensive features

# 1. Average yards per play
avg_yards_per_play_home = all_data.groupby('Home')['Yards'].mean()
avg_yards_per_play_visitor = all_data.groupby('Visitor')['Yards'].mean()
overall_avg_yards_per_play = (avg_yards_per_play_home + avg_yards_per_play_visitor) / 2

# 2. Average total yards per game
total_yards_per_game_home = all_data.groupby(['SeasonYear', 'Home'])['Yards'].sum() / all_data.groupby(['SeasonYear', 'Home']).size()
total_yards_per_game_visitor = all_data.groupby(['SeasonYear', 'Visitor'])['Yards'].sum() / all_data.groupby(['SeasonYear', 'Visitor']).size()
overall_avg_yards_per_game = (total_yards_per_game_home + total_yards_per_game_visitor).groupby(level=1).mean()

# 3. Average pass completion rate
avg_pass_completion_rate_home = all_data.groupby('Home').apply(lambda x: 1 - x['IsIncomplete'].mean())
avg_pass_completion_rate_visitor = all_data.groupby('Visitor').apply(lambda x: 1 - x['IsIncomplete'].mean())
overall_avg_pass_completion_rate = (avg_pass_completion_rate_home + avg_pass_completion_rate_visitor) / 2

# 4. Average touchdowns per game
avg_touchdowns_per_game_home = all_data.groupby(['SeasonYear', 'Home'])['IsTouchdown'].sum() / all_data.groupby(['SeasonYear', 'Home']).size()
avg_touchdowns_per_game_visitor = all_data.groupby(['SeasonYear', 'Visitor'])['IsTouchdown'].sum() / all_data.groupby(['SeasonYear', 'Visitor']).size()
overall_avg_touchdowns_per_game = (avg_touchdowns_per_game_home + avg_touchdowns_per_game_visitor).groupby(level=1).mean()

# 5. Average rush success rate
avg_rush_success_rate_home = all_data.groupby('Home').apply(lambda x: x['Yards'][x['IsRush'] == 1].mean())
avg_rush_success_rate_visitor = all_data.groupby('Visitor').apply(lambda x: x['Yards'][x['IsRush'] == 1].mean())
overall_avg_rush_success_rate = (avg_rush_success_rate_home + avg_rush_success_rate_visitor) / 2

# Creating a dataframe for the new offensive features
new_offensive_features = pd.DataFrame({
    'Team': team_features_combined['Team'],
    'AvgYardsPerPlay': overall_avg_yards_per_play.values,
    'AvgYardsPerGame': overall_avg_yards_per_game.values,
    'AvgPassCompletionRate': overall_avg_pass_completion_rate.values,
    'AvgTouchdownsPerGame': overall_avg_touchdowns_per_game.values,
    'AvgRushSuccessRate': overall_avg_rush_success_rate.values
})

# Merging with the existing combined features
team_features_expanded = team_features_combined.merge(new_offensive_features, on='Team')

team_features_expanded.head()

C:\Users\shash\AppData\Local\Temp\ipykernel_2008\23167019.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_pass_completion_rate_home = all_data.groupby('Home').apply(lambda x: 1 - x['IsIncomplete'].mean())
C:\Users\shash\AppData\Local\Temp\ipykernel_2008\23167019.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_pass_completion_rate_visitor = all_data.groupby('Visitor').apply(lambda x: 1 - x[

Team  AvgPointsScored  AvgPointsAllowed   WinRate  AvgPointsDefended  \
0  ARI        22.000000         25.355769  0.260526          25.355769   
1  ATL        15.492551         19.294365  0.500675          19.294365   
2  BAL        24.633973         14.814507  0.713392          14.814507   
3  BUF        34.659834         13.942320  0.739255          13.942320   
4  CAR        17.259582         25.820934  0.000000          25.820934   

   AvgConcededPlays  AvgForcedTurnovers  AvgYardsPerPlay  AvgYardsPerGame  \
0         -1.011919            0.013292         3.954684         7.909368   
1         -1.017538            0.018890         3.375155         6.750310   
2         -1.021287            0.021287         3.368954         6.737907   
3         -1.030072            0.031484         4.145710         8.291420   
4         -1.018031            0.020741         3.490008         6.980017   

   AvgPassCompletionRate  AvgTouchdownsPerGame  AvgRushSuccessRate  
0               0.884338              0.057776            4.984774  
1               0.867746              0.040468            4.482234  
2               0.883340              0.040894            4.483082  
3               0.906960              0.066286            5.679096  
4               0.867477              0.049143            4.736291

In [35]:
# Calculate additional defensive features

# 1. Average yards allowed per play
avg_yards_allowed_per_play_home = all_data.groupby('Home')['Yards'].mean()
avg_yards_allowed_per_play_visitor = all_data.groupby('Visitor')['Yards'].mean()
overall_avg_yards_allowed_per_play = (avg_yards_allowed_per_play_home + avg_yards_allowed_per_play_visitor) / 2

# 2. Average total yards allowed per game
total_yards_allowed_per_game_home = all_data.groupby(['SeasonYear', 'Home'])['Yards'].sum() / all_data.groupby(['SeasonYear', 'Home']).size()
total_yards_allowed_per_game_visitor = all_data.groupby(['SeasonYear', 'Visitor'])['Yards'].sum() / all_data.groupby(['SeasonYear', 'Visitor']).size()
overall_avg_yards_allowed_per_game = (total_yards_allowed_per_game_home + total_yards_allowed_per_game_visitor).groupby(level=1).mean()

# 3. Average pass completion allowed rate
avg_pass_completion_allowed_rate_home = all_data.groupby('Home').apply(lambda x: 1 - x['IsIncomplete'].mean())
avg_pass_completion_allowed_rate_visitor = all_data.groupby('Visitor').apply(lambda x: 1 - x['IsIncomplete'].mean())
overall_avg_pass_completion_allowed_rate = (avg_pass_completion_allowed_rate_home + avg_pass_completion_allowed_rate_visitor) / 2

# 4. Average touchdowns allowed per game
avg_touchdowns_allowed_per_game_home = all_data.groupby(['SeasonYear', 'Home'])['IsTouchdown'].sum() / all_data.groupby(['SeasonYear', 'Home']).size()
avg_touchdowns_allowed_per_game_visitor = all_data.groupby(['SeasonYear', 'Visitor'])['IsTouchdown'].sum() / all_data.groupby(['SeasonYear', 'Visitor']).size()
overall_avg_touchdowns_allowed_per_game = (avg_touchdowns_allowed_per_game_home + avg_touchdowns_allowed_per_game_visitor).groupby(level=1).mean()

# 5. Average rush success allowed rate
avg_rush_success_allowed_rate_home = all_data.groupby('Home').apply(lambda x: x['Yards'][x['IsRush'] == 1].mean())
avg_rush_success_allowed_rate_visitor = all_data.groupby('Visitor').apply(lambda x: x['Yards'][x['IsRush'] == 1].mean())
overall_avg_rush_success_allowed_rate = (avg_rush_success_allowed_rate_home + avg_rush_success_allowed_rate_visitor) / 2

# Creating a dataframe for the new defensive features
new_defensive_features = pd.DataFrame({
    'Team': team_features_expanded['Team'],
    'AvgYardsAllowedPerPlay': overall_avg_yards_allowed_per_play.values,
    'AvgYardsAllowedPerGame': overall_avg_yards_allowed_per_game.values,
    'AvgPassCompletionAllowedRate': overall_avg_pass_completion_allowed_rate.values,
    'AvgTouchdownsAllowedPerGame': overall_avg_touchdowns_allowed_per_game.values,
    'AvgRushSuccessAllowedRate': overall_avg_rush_success_allowed_rate.values
})

# Merging with the existing combined features
team_features_complete = team_features_expanded.merge(new_defensive_features, on='Team')

team_features_complete

C:\Users\shash\AppData\Local\Temp\ipykernel_2008\2738410617.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_pass_completion_allowed_rate_home = all_data.groupby('Home').apply(lambda x: 1 - x['IsIncomplete'].mean())
C:\Users\shash\AppData\Local\Temp\ipykernel_2008\2738410617.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  avg_pass_completion_allowed_rate_visitor = all_data.groupby('Visitor').ap

Team  AvgPointsScored  AvgPointsAllowed   WinRate  AvgPointsDefended  \
0   ARI        22.000000         25.355769  0.260526          25.355769   
1   ATL        15.492551         19.294365  0.500675          19.294365   
2   BAL        24.633973         14.814507  0.713392          14.814507   
3   BUF        34.659834         13.942320  0.739255          13.942320   
4   CAR        17.259582         25.820934  0.000000          25.820934   
5   CHI        18.665810         34.334604  0.000000          34.334604   
6   CIN        12.227273         23.409632  0.265583          23.409632   
7   CLE        19.919890         18.735727  0.484480          18.735727   
8   DAL        30.742164         10.539742  0.736352          10.539742   
9   DEN        25.289070         38.441368  0.239735          38.441368   
10  DET        26.483073         20.750000  0.748062          20.750000   
11   GB        25.038995         23.928857  0.506477          23.928857   
12  HOU        23.762219         20.104762  0.483204          20.104762   
13  IND        24.123268         24.726269  0.517365          24.726269   
14  JAX        23.583784         20.864865  0.495352          20.864865   
15   KC        25.069551         15.066235  0.742323          15.066235   
16   LA        24.005076         24.229272  0.502545          24.229272   
17  LAC        27.658682         26.215585  0.495609          26.215585   
18   LV        16.380000         24.452381  0.235049          24.452381   
19  MIA        48.566434         26.360140  0.771725          26.360140   
20  MIN        22.711536         24.228278  0.224543          24.228278   
21   NE        13.952487         23.859162  0.261598          23.859162   
22   NO        15.474421         19.042558  0.496774          19.042558   
23  NYG        10.835476         34.491003  0.345550          34.491003   
24  NYJ        13.541156         23.962347  0.239789          23.962347   
25  PHI        29.500000         22.721723  1.000000          22.721723   
26  PIT        15.697082         24.881698  0.511082          24.881698   
27  SEA        31.479419         29.690073  0.687500          29.690073   
28   SF        31.195355         14.487292  1.000000          14.487292   
29   TB        21.114989         16.922872  0.766393          16.922872   
30  TEN        18.124654         17.778393  0.500000          17.778393   
31  WAS        22.435784         29.756359  0.507060          29.756359   

    AvgConcededPlays  AvgForcedTurnovers  AvgYardsPerPlay  AvgYardsPerGame  \
0          -1.011919            0.013292         3.954684         7.909368   
1          -1.017538            0.018890         3.375155         6.750310   
2          -1.021287            0.021287         3.368954         6.737907   
3          -1.030072            0.031484         4.145710         8.291420   
4          -1.018031            0.020741         3.490008         6.980017   
5          -1.019208            0.023257         3.998670         7.997340   
6          -1.019241            0.019241         3.427839         6.855679   
7          -1.020230            0.025280         3.282982         6.565964   
8          -1.021265            0.026639         3.528267         7.056535   
9          -1.014579            0.015895         4.393474         8.786947   
10         -1.014223            0.016807         3.767808         7.535617   
11         -1.012947            0.014245         3.465086         6.930172   
12         -1.019390            0.020685         3.720154         7.440308   
13         -1.022824            0.025222         3.630419         7.260839   
14         -1.023437            0.026863         3.756156         7.512312   
15         -1.025355            0.026692         4.072121         8.144242   
16         -1.017025            0.017025         3.826396         7.652792   
17         -1.020108            0.021370         4.162992         8.325984   
18         -1.022690            0.022690     

In [39]:
# Reload the upcoming games data
upcoming_games = pd.read_csv("BEST New_Upcoming_Schedule.csv")

# Feature encoding: merging the upcoming games data with the team features data
upcoming_encoded_home = upcoming_games.merge(team_features_complete, left_on='Home', right_on='Team', how='left')
upcoming_encoded_both = upcoming_encoded_home.merge(team_features_complete, left_on='Visitor', right_on='Team', suffixes=('_Home', '_Visitor'), how='left')

In [41]:
upcoming_encoded_both

Home Visitor Team_Home  AvgPointsScored_Home  AvgPointsAllowed_Home  \
0   CHI     WAS       CHI             18.665810              34.334604   
1   HOU     ATL       HOU             23.762219              20.104762   
2   CAR     DET       CAR             17.259582              25.820934   
3    NO      NE        NO             15.474421              19.042558   
4   CIN     ARI       CIN             12.227273              23.409632   
5   NYJ     DEN       NYJ             13.541156              23.962347   
6   DAL      SF       DAL             30.742164              10.539742   
7   JAX     BUF       JAX             23.583784              20.864865   
8   TEN     IND       TEN             18.124654              17.778393   
9   NYG     MIA       NYG             10.835476              34.491003   
10  BAL     PIT       BAL             24.633973              14.814507   
11  PHI      LA       PHI             29.500000              22.721723   
12   KC     MIN        KC             25.069551              15.066235   
13   GB      LV        GB             25.038995              23.928857   

    WinRate_Home  AvgPointsDefended_Home  AvgConcededPlays_Home  \
0       0.000000               34.334604              -1.019208   
1       0.483204               20.104762              -1.019390   
2       0.000000               25.820934              -1.018031   
3       0.496774               19.042558              -1.019450   
4       0.265583               23.409632              -1.019241   
5       0.239789               23.962347              -1.022022   
6       0.736352               10.539742              -1.021265   
7       0.495352               20.864865              -1.023437   
8       0.500000               17.778393              -1.022161   
9       0.345550               34.491003              -1.024163   
10      0.713392               14.814507              -1.021287   
11      1.000000               22.721723              -1.022949   
12      0.742323               15.066235              -1.025355   
13      0.506477               23.928857              -1.012947   

    AvgForcedTurnovers_Home  AvgYardsPerPlay_Home  ...  \
0                  0.023257              3.998670  ...   
1                  0.020685              3.720154  ...   
2                  0.020741              3.490008  ...   
3                  0.019450              3.176105  ...   
4                  0.019241              3.427839  ...   
5                  0.024681              3.442188  ...   
6                  0.026639              3.528267  ...   
7                  0.026863              3.756156  ...   
8                  0.022161              3.317175  ...   
9                  0.026881              3.216616  ...   
10                 0.021287              3.368954  ...   
11                 0.025499              3.940535  ...   
12                 0.026692              4.072121  ...   
13                 0.014245              3.465086  ...   

    AvgYardsPerPlay_Visitor  AvgYardsPerGame_Visitor  \
0                  3.558549                 7.117099   
1                  3.375155                 6.750310   
2                  3.767808                 7.535617   
3                  3.517884                 7.035767   
4                  3.954684                 7.909368   
5                  4.393474                 8.786947   
6                  3.852099                 7.704198   
7                  4.145710                 8.291420   
8                  3.630419                 7.260839   
9                  5.139113                10.278227   
10                 3.784394                 7.568789   
11                 3.826396                 7.652792   
12                 4.161101                 8.322201   
13                 3.744752                 7.489504   

    AvgPassCompletionRate_Visitor  AvgTouchdownsPerGame_Visitor  \
0                        0.873646                      0.055276   
1                        0.867746       

In [43]:
for col in ['AvgPointsScored', 'AvgPointsAllowed', 'WinRate', 'AvgPointsDefended', 'AvgConcededPlays', 'AvgForcedTurnovers',
            'AvgYardsPerPlay', 'AvgYardsPerGame', 'AvgPassCompletionRate', 'AvgTouchdownsPerGame', 'AvgRushSuccessRate',
            'AvgYardsAllowedPerPlay', 'AvgYardsAllowedPerGame', 'AvgPassCompletionAllowedRate', 'AvgTouchdownsAllowedPerGame', 'AvgRushSuccessAllowedRate']:
    upcoming_encoded_both[f'Diff_{col}'] = upcoming_encoded_both[f'{col}_Home'] - upcoming_encoded_both[f'{col}_Visitor']

# Selecting only the difference columns and the teams for clarity
upcoming_encoded_final = upcoming_encoded_both[['Home', 'Visitor'] + [col for col in upcoming_encoded_both.columns if 'Diff_' in col]]

upcoming_encoded_final

Home Visitor  Diff_AvgPointsScored  Diff_AvgPointsAllowed  Diff_WinRate  \
0   CHI     WAS             -3.769975               4.578245     -0.507060   
1   HOU     ATL              8.269668               0.810397     -0.017471   
2   CAR     DET             -9.223491               5.070934     -0.748062   
3    NO      NE              1.521934              -4.816605      0.235176   
4   CIN     ARI             -9.772727              -1.946138      0.005056   
5   NYJ     DEN            -11.747914             -14.479022      0.000054   
6   DAL      SF             -0.453191              -3.947550     -0.263648   
7   JAX     BUF            -11.076051               6.922545     -0.243903   
8   TEN     IND             -5.998614              -6.947875     -0.017365   
9   NYG     MIA            -37.730958               8.130863     -0.426175   
10  BAL     PIT              8.936891             -10.067191      0.202310   
11  PHI      LA              5.494924              -1.507550      0.497455   
12   KC     MIN              2.358015              -9.162043      0.517780   
13   GB      LV              8.658995              -0.523524      0.271428   

    Diff_AvgPointsDefended  Diff_AvgConcededPlays  Diff_AvgForcedTurnovers  \
0                 4.578245              -0.000762                 0.000844   
1                 0.810397              -0.001852                 0.001796   
2                 5.070934              -0.003808                 0.003934   
3                -4.816605              -0.002762                -0.001117   
4                -1.946138              -0.007323                 0.005949   
5               -14.479022              -0.007443                 0.008787   
6                -3.947550              -0.005539                 0.010913   
7                 6.922545               0.006635                -0.004621   
8                -6.947875               0.000663                -0.003061   
9                 8.130863              -0.001001                 0.002844   
10              -10.067191               0.002188                -0.004752   
11               -1.507550              -0.005924                 0.008474   
12               -9.162043               0.000781                -0.002139   
13               -0.523524               0.009743                -0.008445   

    Diff_AvgYardsPerPlay  Diff_AvgYardsPerGame  Diff_AvgPassCompletionRate  \
0               0.440121              0.880241                    0.021714   
1               0.344999              0.689998                    0.013390   
2              -0.277800             -0.555600                   -0.004558   
3              -0.341778             -0.683557                   -0.019339   
4              -0.526845             -1.053690                   -0.031547   
5              -0.951286             -1.902571                   -0.017719   
6              -0.323832             -0.647663                    0.006396   
7              -0.389554             -0.779108                   -0.028459   
8              -0.313245             -0.626490                   -0.005762   
9              -1.922497             -3.844995                   -0.028556   
10             -0.415441             -0.830881                    0.020122   
11              0.114139              0.228279                    0.024810   
12             -0.088979             -0.177959                   -0.033750   
13             -0.279666             -0.559332                   -0.018595   

    Diff_AvgTouchdownsPerGame  Diff_AvgRushSuccessRate  \
0                    0.020729                -0.241698   
1                    0.016407                -0.663932   
2                   -0.010272                 0.762481   
3                   -0.010308                 0.396533   
4                   -0.020235                -0.073450   
5                   -0.050169                -1.132587   
6                   -0.005360                -0.147227   
7                   -0.011

Training Data Preparation

In [45]:
training_encoded_home = all_data.merge(team_features_complete, left_on='Home', right_on='Team', how='left')
# Merge the result with team features for visitor teams
training_encoded_both = training_encoded_home.merge(team_features_complete, left_on='Visitor', right_on='Team', suffixes=('_Home', '_Visitor'), how='left')

# Calculate the difference in features
for col in ['AvgPointsScored', 'AvgPointsAllowed', 'WinRate', 'AvgPointsDefended', 'AvgConcededPlays', 'AvgForcedTurnovers',
            'AvgYardsPerPlay', 'AvgYardsPerGame', 'AvgPassCompletionRate', 'AvgTouchdownsPerGame', 'AvgRushSuccessRate',
            'AvgYardsAllowedPerPlay', 'AvgYardsAllowedPerGame', 'AvgPassCompletionAllowedRate', 'AvgTouchdownsAllowedPerGame', 'AvgRushSuccessAllowedRate']:
    training_encoded_both[f'Diff_{col}'] = training_encoded_both[f'{col}_Home'] - training_encoded_both[f'{col}_Visitor']

# Filtering out the required columns
training_data = training_encoded_both[[col for col in training_encoded_both.columns if 'Diff_' in col]]
training_labels = training_encoded_both['HomeWon']

In [47]:
training_data.head()

Diff_AvgPointsScored  Diff_AvgPointsAllowed  Diff_WinRate  \
0             11.278682               1.763204      0.260560   
1             11.278682               1.763204      0.260560   
2             11.278682               1.763204      0.260560   
3             11.278682               1.763204      0.260560   
4             16.789677             -13.319420      0.474754   

   Diff_AvgPointsDefended  Diff_AvgConcededPlays  Diff_AvgForcedTurnovers  \
0                1.763204               0.002583                -0.001320   
1                1.763204               0.002583                -0.001320   
2                1.763204               0.002583                -0.001320   
3                1.763204               0.002583                -0.001320   
4              -13.319420              -0.004577                 0.006072   

   Diff_AvgYardsPerPlay  Diff_AvgYardsPerGame  Diff_AvgPassCompletionRate  \
0              0.418240              0.836480                   -0.011787   
1              0.418240              0.836480                   -0.011787   
2              0.418240              0.836480                   -0.011787   
3              0.418240              0.836480                   -0.011787   
4              0.010384              0.020767                    0.005341   

   Diff_AvgTouchdownsPerGame  Diff_AvgRushSuccessRate  \
0                   0.016709                 0.249559   
1                   0.016709                 0.249559   
2                   0.016709                 0.249559   
3                   0.016709                 0.249559   
4                   0.006815                 0.651589   

   Diff_AvgYardsAllowedPerPlay  Diff_AvgYardsAllowedPerGame  \
0                     0.418240                     0.836480   
1                     0.418240                     0.836480   
2                     0.418240                     0.836480   
3                     0.418240                     0.836480   
4                     0.010384                     0.020767   

   Diff_AvgPassCompletionAllowedRate  Diff_AvgTouchdownsAllowedPerGame  \
0                          -0.011787                          0.016709   
1                          -0.011787                          0.016709   
2                          -0.011787                          0.016709   
3                          -0.011787                          0.016709   
4                           0.005341                          0.006815   

   Diff_AvgRushSuccessAllowedRate  
0                        0.249559  
1                        0.249559  
2                        0.249559  
3                        0.249559  
4                        0.651589

In [49]:
training_data.head()

Diff_AvgPointsScored  Diff_AvgPointsAllowed  Diff_WinRate  \
0             11.278682               1.763204      0.260560   
1             11.278682               1.763204      0.260560   
2             11.278682               1.763204      0.260560   
3             11.278682               1.763204      0.260560   
4             16.789677             -13.319420      0.474754   

   Diff_AvgPointsDefended  Diff_AvgConcededPlays  Diff_AvgForcedTurnovers  \
0                1.763204               0.002583                -0.001320   
1                1.763204               0.002583                -0.001320   
2                1.763204               0.002583                -0.001320   
3                1.763204               0.002583                -0.001320   
4              -13.319420              -0.004577                 0.006072   

   Diff_AvgYardsPerPlay  Diff_AvgYardsPerGame  Diff_AvgPassCompletionRate  \
0              0.418240              0.836480                   -0.011787   
1              0.418240              0.836480                   -0.011787   
2              0.418240              0.836480                   -0.011787   
3              0.418240              0.836480                   -0.011787   
4              0.010384              0.020767                    0.005341   

   Diff_AvgTouchdownsPerGame  Diff_AvgRushSuccessRate  \
0                   0.016709                 0.249559   
1                   0.016709                 0.249559   
2                   0.016709                 0.249559   
3                   0.016709                 0.249559   
4                   0.006815                 0.651589   

   Diff_AvgYardsAllowedPerPlay  Diff_AvgYardsAllowedPerGame  \
0                     0.418240                     0.836480   
1                     0.418240                     0.836480   
2                     0.418240                     0.836480   
3                     0.418240                     0.836480   
4                     0.010384                     0.020767   

   Diff_AvgPassCompletionAllowedRate  Diff_AvgTouchdownsAllowedPerGame  \
0                          -0.011787                          0.016709   
1                          -0.011787                          0.016709   
2                          -0.011787                          0.016709   
3                          -0.011787                          0.016709   
4                           0.005341                          0.006815   

   Diff_AvgRushSuccessAllowedRate  
0                        0.249559  
1                        0.249559  
2                        0.249559  
3                        0.249559  
4                        0.651589

In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Initialize the logistic regression model
logreg = LogisticRegression(max_iter=1000)

# Evaluate the model's performance using cross-validation
cross_val_scores = cross_val_score(logreg, training_data, training_labels, cv=5)

cross_val_scores_mean = cross_val_scores.mean()

cross_val_scores_mean

0.75302023407538

In [53]:
# Checking the shape of the training data
training_data.shape

(12009, 16)

In [55]:
nan_columns = training_data.columns[training_data.isna().any()].tolist()

# Displaying columns with NaN values and the number of NaN values in them
nan_counts = training_data[nan_columns].isna().sum()
nan_counts


Series([], dtype: float64)

In [57]:
training_data_cleaned = training_data.dropna()
training_labels_cleaned = training_labels.loc[training_data_cleaned.index]

# Checking the shape of the cleaned data
training_data_cleaned.shape, training_labels_cleaned.shape

((12009, 16), (12009,))

In [59]:
cross_val_scores_cleaned = cross_val_score(logreg, training_data_cleaned, training_labels_cleaned, cv=5)

cross_val_scores_cleaned_mean = cross_val_scores_cleaned.mean()

cross_val_scores_cleaned_mean

0.75302023407538

In [61]:
logreg.fit(training_data_cleaned, training_labels_cleaned)

LogisticRegression(max_iter=1000)

Make Predictions On Upcoming Games

In [64]:
upcoming_game_probabilities = logreg.predict_proba(upcoming_encoded_final[[col for col in upcoming_encoded_final.columns if 'Diff_' in col]])

In [66]:
upcoming_game_probabilities

array([[0.95763121, 0.04236879],
       [0.55390523, 0.44609477],
       [0.99491747, 0.00508253],
       [0.22328809, 0.77671191],
       [0.39716048, 0.60283952],
       [0.41028229, 0.58971771],
       [0.8987413 , 0.1012587 ],
       [0.61175983, 0.38824017],
       [0.46961497, 0.53038503],
       [0.75684827, 0.24315173],
       [0.39386212, 0.60613788],
       [0.03464301, 0.96535699],
       [0.04173709, 0.95826291],
       [0.22265833, 0.77734167]])

In [68]:
# Extract the probability that the home team will win (second column of the result)
upcoming_game_prob_home_win = upcoming_game_probabilities[:, 1]

# Add the predictions to the upcoming games dataframe
upcoming_encoded_final['HomeWinProbability'] = upcoming_game_prob_home_win

# Sort by the probability of the home team winning for better visualization
upcoming_predictions = upcoming_encoded_final[['Home', 'Visitor', 'HomeWinProbability']].sort_values(by='HomeWinProbability', ascending=False)

upcoming_predictions

C:\Users\shash\AppData\Local\Temp\ipykernel_2008\2773497604.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upcoming_encoded_final['HomeWinProbability'] = upcoming_game_prob_home_win


Home Visitor  HomeWinProbability
11  PHI      LA            0.965357
12   KC     MIN            0.958263
13   GB      LV            0.777342
3    NO      NE            0.776712
10  BAL     PIT            0.606138
4   CIN     ARI            0.602840
5   NYJ     DEN            0.589718
8   TEN     IND            0.530385
1   HOU     ATL            0.446095
7   JAX     BUF            0.388240
9   NYG     MIA            0.243152
6   DAL      SF            0.101259
0   CHI     WAS            0.042369
2   CAR     DET            0.005083